<a href="https://colab.research.google.com/github/agroimpacts/nmeo/blob/class%2Ff2023/materials/code/notebooks/gee_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Earth Engine using `colab` and `python`

Earth Engine can also accessed through several python client libraries, either from your local computer or through web-platforms, such as Google Colab. We are going to pick up from the code editor examples we worked through since last class, and learn how to run them from `colab`/`python`.

## Import and authenticate the Earth Engine Python API

Along with the [`geemap`](https://geemap.org/) library.

In [1]:
import ee
import geemap
import math

In [ ]:
# Authenticate/Initialize the Earth Engine API
ee.Authenticate()
ee.Initialize()

## SRTM DEM (and derivative data) for Zambia

We are going to condense the work in slides 3 a bit here, by:

1. Loading in the SRTM
2. Calculating slope, aspect, etc
3. Calculate Zambia's mean elevation

### Steps 1 and 2

In [3]:
# Load in SRTM
srtm = ee.Image('CGIAR/SRTM90_V4')

# Load in Zambia shape
zam = ee.Feature(
    ee.FeatureCollection("USDOS/LSIB/2013")\
      .filterMetadata('cc', 'equals', 'ZA')\
      .first()
)

# Slope, aspect, sin of aspect
slope = ee.Terrain.slope(srtm)
aspect = ee.Terrain.aspect(srtm)
sin_aspect = aspect.divide(180).multiply(math.pi).sin()

### Step 3

In [4]:
def mean_dict(image, geometry=zam.geometry(),
              scale=90, maxpixels=1e8):
  reduced = image.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=geometry,
      scale=scale,
      maxPixels=maxpixels,
      bestEffort=True
  )
  return reduced.getInfo()

mean_dict(srtm)

{'elevation': 1120.5801240535598}

### Let's make a map

The code below uses `geemap` to display elevation, slope, aspect, the sin of aspect (eastness), as well as the borders of Zambia, and a little AOI.

These are currently presented in just default gray-scale palettes.  

**Add the code needed to make them have a blue-green-red palette as we did with the code editor**

In [ ]:
# Create an AOI
myaoi = ee.Geometry.Rectangle([30, -12.5, 30.5, -12])

# Create an instance of geemap
map = geemap.Map()
map.setCenter(28, -13, 6)
map.addLayer(srtm, {}, 'SRTM DEM')
map.addLayer(slope, vis_params={min: 0, max: 60}, name="Slope")
map.addLayer(aspect, {}, "Aspect")
map.addLayer(sin_aspect, {}, "Sin of aspect")
map.addLayer(zam, {}, 'Zambia Borders')
map.addLayer(myaoi, {'color': "red"}, "My AOI")
map

## Filter a Landsat Collection

Using the AOI we made above, spatially filter the Landsat 8 collection, then temporally filter it.

In [22]:
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')

# Spatial filter
spatial_filtered = l8.filterBounds(myaoi)
print('spatial_filtered', spatial_filtered.getInfo())

temporal_filtered = spatial_filtered\
  .filterDate('2018-05-01', '2018-09-15');
print('temporal_filtered', temporal_filtered.getInfo());


spatial_filtered {'type': 'ImageCollection', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B7', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B8', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B9', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B10', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B11', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'BQA', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}}], 'id': 'LANDSAT/LC08/C01/T1_TOA', 'version': 16553375494

### Display just one of the images
By index

In [34]:
# Index function
def display_image(collection, index, vis_params, layer_name, map):
  collection_list = collection.toList(collection.size())
  scene = ee.Image(collection_list.get(index))
  map.addLayer(scene, vis_params, layer_name)
  return map


In [ ]:
vis_params = {"bands": ['B4', 'B3', 'B2'], "max": 0.3}
map = geemap.Map()
map.setCenter(28, -13, 6)
display_image(temporal_filtered, 0, vis_params, "First image", map)

### Filter by cloudiness, take median, mask, crop, etc

In [42]:
# Cloudiness
sorted = temporal_filtered.sort('CLOUD_COVER')

# Get the first (least cloudy) image.
scene = ee.Image(sorted.first())
print('Least Cloudy', scene.getInfo())

# Median image composite
median = temporal_filtered.median()

# Mask by elevation
elevmask = srtm.gt(900);
masked_median = median.updateMask(elevmask)

# Crop to AOI
median_crop = median.clip(myaoi)

Least Cloudy {'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7571, 7711], 'crs': 'EPSG:32636', 'crs_transform': [30, 0, 90885, 0, -30, -1164885]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7571, 7711], 'crs': 'EPSG:32636', 'crs_transform': [30, 0, 90885, 0, -30, -1164885]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7571, 7711], 'crs': 'EPSG:32636', 'crs_transform': [30, 0, 90885, 0, -30, -1164885]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7571, 7711], 'crs': 'EPSG:32636', 'crs_transform': [30, 0, 90885, 0, -30, -1164885]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7571, 7711], 'crs': 'EPSG:32636', 'crs_transform': [30, 0, 90885, 0, -30, -1164885]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7571, 7711], 

Map

In [ ]:
map = geemap.Map()
map.setCenter(28, -13, 8)
map.addLayer(scene, vis_params, 'Least cloudy')
map.addLayer(median, vis_params, "Median composite")
map.addLayer(masked_median, vis_params, "Masked Median")
map

## Derive NDVI

In [47]:
# NDVI function
def add_ndvi(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

def add_ndvi_and_crop(image, cropshape=myaoi):
  img = image.clip(myaoi)
  ndvi = img.normalizedDifference(['B5', 'B4']).rename('NDVI')
  return img.addBands(ndvi)

# Apply function to single image
ndvi = add_ndvi(scene).select('NDVI')

# Map onto a collection
with_ndvi = temporal_filtered.map(add_ndvi)

# Applying NDVI to collection and clip
zam_ndvi = temporal_filtered.map(add_ndvi_and_crop)


In [ ]:
vis_params = {"bands": ['NDVI'], "max": 1}
map = geemap.Map()
map.setCenter(30, -12, 8)
map.addLayer(ndvi, vis_params, 'Least cloudy')
display_image(with_ndvi, 0, vis_params, "First uncropped NDVI image", map)
display_image(zam_ndvi, 0, vis_params, "First cropped NDVI image", map)
map

## Cloud masking and LAI calculation

In [58]:
def mask_l8(image):
  qa = image.select('BQA')
  # Check that the cloud bit is off.
  # See https://www.usgs.gov/land-resources/nli/landsat/landsat-collection-1-level-1-quality-assessment-band
  mask = qa.bitwiseAnd(1 << 4).eq(0)
  return image.updateMask(mask)

# apply mask
with_ndvi_masked = with_ndvi.map(mask_l8)

# Converting to LAI
# https://developers.google.com/earth-engine/guides/image_math
# compute LAI and add it as a separate band
def add_lai_ndvi(image):
  lai = image.expression(
    '(log(-(ndvi - 0.943)/0.731))/(log(0.6))',
     {'ndvi': image.select('NDVI')})\
     .rename('LAI_NDVI')
  return(image.addBands(lai))

with_lai = with_ndvi_masked.map(add_lai_ndvi)

# Find Max NDVI/LAI
with_lai_max = with_lai.max()

In [ ]:
# add code to display LAI max
vis_params = {"bands": ['LAI_NDVI'], max: 25}
map = geemap.Map()
map.setCenter(30, -12, 8)
display_image(with_lai, 0, vis_params, "First uncropped NDVI image", map)
map.addLayer(with_lai_max, vis_params, 'L8 LAI - SR')
map

## On your own

1. Create the time series chart in the javascript code
2. Export the scene as in the javascript code